### Importing Libraries

In [19]:
import tensorflow as tf
tf.compat.v1.set_random_seed(0)
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.models import Model
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib as plt
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

### Number of Classes

In [20]:
num_classes=38

### Function to Plot images After conversion to pixels (Float32)

In [21]:
def show_data(sample):
    """
    Shows the given sample data as an image
    """
    import matplotlib.pyplot as plt
    plt.imshow(sample)
    plt.show()

### Loading the Dataset

In [22]:
train_folder_path = 'D:/Coding/Python/Deep learning Project/color'
# Get the list of class (species) folders
train_class_folders = glob(train_folder_path + '/*')

# Initialize lists to store images and labels
images = []
labels = []


# Load images and labels from subfolders
for class_folder in train_class_folders:
    # Extract class (species) label from the folder name
    label = (class_folder.split('\\')[-1])  # Adjust the index based on your folder structure

    # Get the list of image file paths in the current class folder
    image_files = glob(class_folder + '/*.jpg')  # Update the extension based on your image format

    # Load and resize images
    for image_file in image_files:
        img = cv2.imread(image_file)
        img = cv2.resize(img, (64, 64))  # Adjust the size as needed
        images.append(img)
        labels.append(label)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Optionally, normalize pixel values to be in the range [0, 1]
images = images.astype('float32') / 255.0

# Print the shape of the data
print("x_train shape:", images.shape)
print("y_train shape:", labels.shape)


x_train shape: (0,)
y_train shape: (0,)


In [35]:
show_data(images[0])

IndexError: index 0 is out of bounds for axis 0 with size 0

### Input Size Dimensions

In [24]:
input_shape = images.shape[1:]
print(input_shape)

()


### Samples Size

In [25]:
print(images.shape[0], 'train samples')
print(labels.shape[0], 'test samples')
print('x_train shape:', images.shape)
print('y_train shape:', labels.shape)

0 train samples
0 test samples
x_train shape: (0,)
y_train shape: (0,)


### Function of Single Resnet Layer

In [26]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

### Resnet Model

In [27]:
def resnet_v2(input_shape, depth, num_classes=38):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

### Encoding Categorical Values

In [28]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Label encoding
label_encoder = LabelEncoder()
labels1 = label_encoder.fit_transform(labels)

# One-hot encoding
labels = to_categorical(labels1,num_classes)

# Display the results
print("Integer Encoded Labels:", labels1)
print("One-Hot Encoded Labels:\n", labels)


Integer Encoded Labels: []
One-Hot Encoded Labels:
 []


### Splitting Dataset into Training and Validation

In [30]:
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.15, random_state=42)

ValueError: With n_samples=0, test_size=0.15 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

### Specifying Model Parameters

In [31]:
n=3
depth = n*9+2
model = resnet_v2(input_shape=input_shape,depth=depth)
batch_size = 38  # orig paper trained all networks with batch_size=128
epochs = 10

ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=1. Full shape received: (None,)

### Compiling Model

In [32]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=['accuracy'])
model.summary()

NameError: name 'model' is not defined

### Training the Model

In [33]:
model1=model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_val, y_val))

NameError: name 'model' is not defined

### Plotting the loss and accuracy metrics

In [34]:
plt.figure(figsize = (20,5))
plt.subplot(1,2,1)
plt.title("Train and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.plot(model1.history['loss'],label="Train Loss")
plt.plot(model1.history['val_loss'], label="Validation Loss")
plt.xlim(0, 10)
plt.ylim(0.0,1.0)
plt.legend()

plt.subplot(1,2,2)
plt.title("Train and Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.plot(model1.history['accuracy'], label="Train Accuracy")
plt.plot(model1.history['val_accuracy'], label="Validation Accuracy")
plt.xlim(0, 9.25)
plt.ylim(0.75,1.0)
plt.legend()
plt.tight_layout()

AttributeError: module 'matplotlib' has no attribute 'figure'